In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 29 19:21:50 2020

@author: Anqi
"""
from datetime import datetime
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
from re import sub
from lxml import html
import requests
from requests.exceptions import ConnectionError
#import unicodecsv as csv
import csv
import json
import re
import time
import random
from random import randint
import ast
import pandas as pd


C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (2.0.3)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
user_agent_dict = {'Mozilla/5.0':1,
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15':2,
             'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/55.0':3,
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36':4,
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0':5,
             'Mozilla/5.0 (Windows NT 9.6; Win64; x64; rv:78.3) AppleWebKit/673.39 (KHTML, like Gecko) Chrome/83.0.332.97 Safari/537.36':6,
             'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36':7,
             'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36':8,
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25':9,
             'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0':10,
             'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36':11,
             'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10':12,
             'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko':13,
             'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0':14,
             'Mozilla/5.0 (Windows NT 8.2; WOW64) AppleWebKit/606.1.12 (KHTML, like Gecko) Chrome/82.0.2125.104 Safari/537.36':15,
             'Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)':16,
             'Mozilla/5.0 (Windows NT 7.2; Win64; x64; rv:16.0.1) Gecko/20161011 Firefox/16.0.1':17,
             'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25':18
            }

In [3]:
def clean(text):
    if text:
        return ' '.join(' '.join(text).split())
    return None

In [4]:
def rand_agent():
    user_agent = [x for x in user_agent_dict.keys()]
    return random.choice(user_agent)

In [5]:
def get_headers():
    # Creating headers.
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'
               }
    return headers

In [6]:
def create_url(state, county_city_zipcode, pricetype):
    # Creating Zillow URL based on the filter.
    if pricetype == 'sold':
        # for sale
        url = 'https://www.zillow.com/{0}-{1}/sold/'.format(county_city_zipcode, state)
    else:
        # for rent
        url = 'https://www.zillow.com/homes/for_rent/{}/0_singlestory/pricea_sort/'.format(county_city_zipcode)
    print(url)
    return url

In [7]:
def save_to_file(response):
    # saving response to `response.html`
    with open("response.html", 'w', encoding="utf-8") as fp:
        fp.write(response.text)


In [37]:
def write_data_to_csv(data, county_city_zipcode, state, pricetype):
    # saving scraped data to csv.

    with open("C:/Users/Administrator/Desktop/GS_zillow1/zillow_historical_data_zipcode/%s-price-%s-%s-%s.csv" 
              % (pricetype, county_city_zipcode, state, datetime.today().strftime('%Y-%m-%d')), 'w', newline='') as csvfile:

        fieldnames = ['zpid', 'type', 'title', 'address', 'city', 'state', 'zipcode', 
                      'facts and features', 'price', 'bedrooms', 'bathrooms', 'sqft',
                      'lot_size', 'lot_area_unit', 'year_built', 'price_history', 'tax_history', 
                      'latlong', 'property_url', 'photo_main', 'is_published', 'list_date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [38]:
def write_historical_data_to_csv(data, county_city_zipcode, state, price_or_tax, pricetype):
    # saving scraped data to csv.

    with open("C:/Users/Administrator/Desktop/GS_zillow1/zillow_historical_data_zipcode/%s-%s-history-%s-%s-%s.csv" % (pricetype, price_or_tax, county_city_zipcode, state, datetime.today().strftime('%Y-%m-%d')), 'w', newline='') as csvfile:

        if price_or_tax == 'price':
            fieldnames = ['zpid', 'date', 'event', 'price']
        else:
            fieldnames = ['zpid', 'date', 'taxPaid']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [10]:
def get_response(url):
    # Getting response from zillow.com.

    for i in range(5):
        response = requests.get(url, headers=get_headers())
        print("status code received:", response.status_code)
        if response.status_code != 200:
            # saving response to file for debugging purpose.
            save_to_file(response)
            continue
        else:
            save_to_file(response)
            return response
    return None

In [11]:
def find_valid_parentheses_index(text):
    # find the index of '}' that make the text start with '{' a valid parenthesis
    # input: a text start with '{'
    # return type: integer
    count = 0
    for i in range(len(text)):
        if text[i] == '{':
            count += 1
        elif text[i] == '}':
            count -= 1
        else:
            continue
        if count == 0:
            return i

In [12]:
def get_price_history(home_detail_data):
    # input: a long string contains price history
    # return Type: a list consis of dictionaries as records of price history
    try:
        startpattern = '\\"priceHistory\\":'
        endpattern = ']'
        start = home_detail_data.find(startpattern) + len(startpattern)
        end = home_detail_data[start:].find(endpattern) + start + 1
        pricehiststr = home_detail_data[start:end]
        result = re.sub(r"\\", "", pricehiststr, flags=re.I)
        resultjson = json.loads(result)
        #keylist = ['time','event','price','source']
        keylist = ['time','event','price']
        price_history = []
        for history in resultjson:
            record = {x : history[x] if x!='time' 
                      else datetime.fromtimestamp(history[x] / 1e3).strftime('%Y-%m-%d') for x in keylist}
            price_history.append(record)
        return price_history
    except:
        print("Can't find price history.")
        return None
#        return [{'time': '','price': None,'source': '','event': ''}]

In [13]:
def get_tax_history(home_detail_data):
    # input: a long string contains tax history
    # return Type: a list consis of dictionaries as records of tax history
    try:
        startpattern = '\\"taxHistory\\":'
        endpattern = ']'
        start = home_detail_data.find(startpattern) + len(startpattern)
        end = home_detail_data[start:].find(endpattern) + start + 1
        taxhiststr = home_detail_data[start:end]
        result = re.sub(r"\\", "", taxhiststr, flags=re.I)
        resultjson = json.loads(result)
        keylist = ['time','taxPaid']
        tax_history = []
        for history in resultjson:
            record = {x : history[x] if x!='time' 
                      else datetime.fromtimestamp(history[x] / 1e3).strftime('%Y') for x in keylist}
            tax_history.append(record)
        return tax_history
    except:
        print("Can't find tax history.")
        return None
#        return [{'time': '','tax': None}]

In [14]:
def get_home_facts_and_features(home_detail_data):
    # input: a long string contains facts and features
    # return Type: a dictionary of facts and features
    try:
        startpattern = '{\\"property\\":'
        #endpattern = ',\\"abTests\\":'
        start = home_detail_data.find(startpattern) + len(startpattern)
        end = find_valid_parentheses_index(home_detail_data[start:]) + start + 1
        featuresstr = home_detail_data[start:end]
        result = re.sub(r"\\", "", featuresstr, flags=re.I)
        resultjson = json.loads(result)
        features = ['price','yearBuilt','lotAreaUnit']
        featuresjson = {}
        for f in features:
            try:
                featuresjson[f] = resultjson[f]
            except:
                print('No {} Data.'.format(f))
        return featuresjson
    except:
        print("Can't get home details.")
        return {'price': None,'dateSold': None,'yearBuilt': None,'lotAreaUnit': ''}

In [15]:
def get_homedetails(property_url):
    # input: url
    # return Type: dictionary
    for i in range(15):
        random_agent = {'User-Agent': rand_agent()}
        #print("Send request...")
        req = Request(property_url, headers=random_agent)
        try:
            #print("Open webpage...")
            webpage = urlopen(req).read()
            #print("Get webpage..")
        except ConnectionError as exception:
            print('Try to get home details response ({}) time.'.format(i+1))
            print(exception)
            time.sleep(randint(1,5))
            continue
        except ConnectionResetError as exception:
            print('Try to get home details response ({}) time.'.format(i+1))
            print(exception)
            time.sleep(randint(1,5))
            continue
        except URLError as exception:
            print('Try to get home details response ({}) time.'.format(i+1))
            print(exception)
            time.sleep(randint(1,5))
            continue
        parser = html.fromstring(webpage)
        try:
            raw_json_data = parser.get_element_by_id('hdpApolloPreloadedData').text
            agentidx = user_agent_dict[random_agent.get('User-Agent')]
            return raw_json_data, agentidx
        except:
            print('Try to get home details response ({}) time.'.format(i+1))
            try:
                response = get_response(property_url)
                parser = html.fromstring(response.text)
                raw_json_data = parser.get_element_by_id('hdpApolloPreloadedData').text
                return raw_json_data
            except:
                time.sleep(randint(3,10))
                continue
    print("Can't find home details.")
    return None, None


In [16]:
def get_data_from_json(raw_json_data):
    # getting data from json (type 2 of their A/B testing page)
    cleaned_data = clean(raw_json_data).replace('<!--', "").replace("-->", "")
    properties_list = []
    
    try:
        json_data = json.loads(cleaned_data)
        search_results = json_data.get('cat1').get('searchResults').get('listResults', [])
        print(json_data.get('searchPageSeoObject'))
        total_listing = int(json_data.get('cat1').get('searchList').get('totalResultCount'))
        count = 0
        result_size = len(search_results)
        print("how many results are we processing?", result_size)

        valid_agent_lst = [0 for _ in range(len(user_agent_dict))]
        
        for properties in search_results:
            #print("processing result", count, properties)
            print('({}) item of the page'.format(count+1))
            zpid = properties.get('zpid')
            statusText = properties.get('statusText')
            if not zpid.isdigit() or statusText == 'Off market':
                print("bypassing zillow id:", zpid, "statusText:", statusText)
                continue
            address = properties.get('address')
            property_info = properties.get('hdpData', {}).get('homeInfo')
            city = property_info.get('city')
            state = property_info.get('state')
            zipcode = property_info.get('zipcode')
            lot_size = property_info.get('lotAreaValue')
            hometype = property_info.get('homeType')
            try:
                price = int(property_info.get('price'))
            except:
                price = 0
            bedrooms = properties.get('beds')
            bathrooms = properties.get('baths')
            area = properties.get('area')
            property_url = properties.get('detailUrl')
            info = f'{bedrooms} bds,{bathrooms} ba,{area} sqft,{property_url}'
            title = properties.get('statusText')
            photo_main = properties.get('imgSrc')
            latLong = properties.get('latLong')
            #print("Get home details..")
            homedetails,agentidx = get_homedetails(property_url)
            if agentidx:
                valid_agent_lst[agentidx-1] += 1
            home_features = get_home_facts_and_features(homedetails)
            if home_features.get('price'):
                price = home_features.get('price')
            year_built = home_features.get('yearBuilt')
            lot_area_unit = home_features.get('lotAreaUnit')
            price_history = get_price_history(homedetails)
            tax_history = get_tax_history(homedetails)


            data = {'address': address,
                    'city': city,
                    'state': state,
                    'zpid' : zpid,
                    'zipcode': zipcode,
                    'price': price,
                    'bedrooms': bedrooms,
                    'bathrooms': bathrooms,
                    'sqft': area,
                    'year_built': year_built,
                    'lot_size': lot_size,
                    'lot_area_unit': lot_area_unit,
                    'latlong': latLong,
                    'price_history': price_history,
                    'tax_history': tax_history,
                    'facts and features': info,
                    'title': title,
                    'type': hometype,
                    'property_url': property_url,
                    'photo_main': photo_main,
                    'is_published': True, #set this as false for now
                    'list_date': datetime.today().strftime('%Y-%m-%d')}
            properties_list.append(data)

            count += 1
#            if count%10 == 0:
#                time.sleep(randint(3,10))

        return total_listing, properties_list, valid_agent_lst

    except ValueError:
        print("Invalid json", ValueError)
        return None


In [17]:

def parse(state, county_city_zipcode, pricetype):
    url = create_url(state, county_city_zipcode, pricetype)
    url = 'https://www.zillow.com/homes/Jasper,-GA_rb/'    
    response = get_response(url)

    if not response:
        print("Failed to fetch the page, please check `response.html` to see the response received from zillow.com.")
        return None

    for i in range(10):

        random_agent = {'User-Agent': rand_agent()}
        req = Request(url, headers=random_agent)

        # start over the page when catch exception
        try:
            webpage = urlopen(req).read()
        except ConnectionError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except ConnectionResetError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except HTTPError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except URLError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        parser = html.fromstring(webpage)
        raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')

        try:
#            total_listing, prop_list, valid_agent_lst = total_listing, properties_list, valid_agent_lst
            
#             for page in range(16,19):
#                page = 3
            url = 'https://www.zillow.com/homes/Jasper,-GA_rb/'.format(page,page)    
            response = get_response(url)
            random_agent = {'User-Agent': rand_agent()}
            req = Request(url, headers=random_agent)
            webpage = urlopen(req).read()
            parser = html.fromstring(webpage)
            raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
            total_listing, prop_list, valid_agent_lst = get_data_from_json(raw_json_data)
            scraped_data.extend(prop_list)

            ak = []
            for x1,x2 in zip(a1,a2):
                if x1 != x2:
                    ak.append(x1)
            
            
            break
        except TypeError as exception:
            # start over the page
            print(exception)
            continue
        except AttributeError as exception:
            print(exception)
            try:
                response = get_response(url)
                parser = html.fromstring(response.text)
                raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
                total_listing, prop_list, valid_agent_lst = get_data_from_json(raw_json_data)
                break
            except:
                time.sleep(randint(1,5))
                continue
        except HTTPError as exception:
            print(exception)
            try:
                response = get_response(url)
                parser = html.fromstring(response.text)
                raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
                total_listing, prop_list, valid_agent_lst = get_data_from_json(raw_json_data)
                break
            except:
                time.sleep(randint(1,5))
                continue
        except ConnectionError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except ConnectionResetError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except URLError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
    
    temp = valid_agent_lst
    print("Valid User Agent Stats:")
    print(temp)
    print("total number of listing in this area {0} is {1}".format(county_city_zipcode, total_listing))
    page_num = 2
    page_url = "{0}{1}_p/".format(url, page_num)
   
# while (page_num <= 20) and (page_num * 40 <= total_listing or ((page_num-1) * 40 < total_listing)) and requests.get(page_url, headers=get_headers()).status_code == 200:
    while (page_num <= 20) and (page_num * 40 <= total_listing or ((page_num-1) * 40 < total_listing)) and requests.get(page_url, headers=get_headers()).status_code == 200:
        print("reading url:", page_url)
        random_agent = {'User-Agent': rand_agent()}
        req = Request(page_url, headers=random_agent)

         # start over the page when catch exception
        try:
            webpage = urlopen(req).read()
        except ConnectionError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except ConnectionResetError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except HTTPError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        parser = html.fromstring(webpage)
        raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')

        try:
            t, p_list, valid_agent_lst = get_data_from_json(raw_json_data)
        except TypeError as exception:
            # start over the page
            print(exception)
            continue
        except AttributeError as exception:
            # start over the page
            print(exception)
            try:
                response = get_response(page_url)
                parser = html.fromstring(response.text)
                raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
                t, p_list, valid_agent_lst = get_data_from_json(raw_json_data)
            except:
                time.sleep(randint(1,5))
                continue
        except HTTPError as exception:
            print(exception)
            try:
                response = get_response(page_url)
                parser = html.fromstring(response.text)
                raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
                t, p_list, valid_agent_lst = get_data_from_json(raw_json_data)
            except:
                time.sleep(randint(1,5))
                continue
        except ConnectionError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
        except ConnectionResetError as exception:
            print(exception)
            time.sleep(randint(1,5))
            continue
            
        prop_list.extend(p_list)
        page_num += 1
        page_url = "{0}{1}_p/".format(url, page_num)
        valid_agent_lst = [a + b for a, b in zip(valid_agent_lst, temp)]
        temp = valid_agent_lst
        print("Valid User Agent Stats:")
        print(valid_agent_lst)

    return prop_list, valid_agent_lst

In [18]:
def remove_duplicates(scraped_data):
    zpid_list = []
    cleaned_scraped_data = []
    for data in scraped_data:
        if data['zpid'] not in zpid_list:
            zpid_list.append(data['zpid'])
            cleaned_scraped_data.append(data)
    return cleaned_scraped_data

In [19]:
def get_price_tax_history_list(cleaned_scraped_data):
    
    # initializing the fields and price tax list 
    price_history_list = []
    tax_history_list = []
      
    # extracting hitorical data for each house
    for row in cleaned_scraped_data: 
        zpid = row['zpid']
        if not row['price_history']:
            price_history_single = [{'zpid':zpid, 'date':'', 'event':'', 'price':None}]
        elif row['price_history'] == '[]':
            price_history_single = [{'zpid':zpid, 'date':'', 'event':'', 'price':None}]
        else:
            price = row['price_history']
            if type(price) is not list:
                price = [price]
            price_history_single = [{'zpid':zpid, 'date':d['time'], 'event':d['event'], 'price':d['price']} for d in price]

        if not row['tax_history']:
            tax_history_single = [{'zpid':zpid, 'date':'', 'taxPaid':None}]
        elif row['tax_history'] == '[]':
            tax_history_single = [{'zpid':zpid, 'date':'', 'taxPaid':None}]
        else:
            tax = row['tax_history']
            if type(tax) is not list:
                tax = [tax]
            tax_history_single = [{'zpid':zpid, 'date':d['time'], 'taxPaid':d['taxPaid']} for d in tax]
            
        price_history_list.extend(price_history_single)
        tax_history_list.extend(tax_history_single)
            
    return price_history_list, tax_history_list


In [20]:
pricetype = 'sold'
state = 'easley-sc'
county_city_zipcode = '29640'
scraped_data = []
page = 1

In [40]:
######################################################################################################################
# change url for each page
for page in range(1, 6):
    url = 'https://www.zillow.com/easley-sc/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A{page}%7D%2C%22usersSearchTerm%22%3A%22Easley%2C%20SC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.83817456542968%2C%22east%22%3A-82.32319043457031%2C%22south%22%3A34.64722799377236%2C%22north%22%3A35.054067380840195%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A38272%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D'
    response = get_response(url)
    random_agent = {'User-Agent': rand_agent()}
    req = Request(url, headers=random_agent)
    webpage = urlopen(req).read()

    webpage = webpage.decode('utf-8')

    parser = html.fromstring(webpage)
    raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
    total_listing, prop_list, valid_agent_lst = get_data_from_json(raw_json_data)
scraped_data.extend(prop_list)

status code received: 200
{'baseUrl': '/easley-sc/2_p/', 'windowTitle': 'Easley Real Estate - Easley SC Homes For Sale | Zillow'}
how many results are we processing? 40
(1) item of the page
(2) item of the page
No lotAreaUnit Data.
(3) item of the page
No yearBuilt Data.
(4) item of the page
No yearBuilt Data.
No lotAreaUnit Data.
(5) item of the page
No lotAreaUnit Data.
(6) item of the page
Try to get home details response (1) time.
status code received: 200
No lotAreaUnit Data.
(7) item of the page
Try to get home details response (1) time.
status code received: 200
No lotAreaUnit Data.
(8) item of the page
(9) item of the page
(10) item of the page
No lotAreaUnit Data.
(11) item of the page
No lotAreaUnit Data.
(12) item of the page
(13) item of the page
No lotAreaUnit Data.
(14) item of the page
(15) item of the page
No lotAreaUnit Data.
(16) item of the page
No yearBuilt Data.
(17) item of the page
(18) item of the page
Try to get home details response (1) time.
status code recei

(26) item of the page
(27) item of the page
No lotAreaUnit Data.
(28) item of the page
No lotAreaUnit Data.
(29) item of the page
No yearBuilt Data.
(30) item of the page
No yearBuilt Data.
(31) item of the page
Try to get home details response (1) time.
status code received: 200
No lotAreaUnit Data.
(32) item of the page
(33) item of the page
No yearBuilt Data.
(34) item of the page
(35) item of the page
No lotAreaUnit Data.
(36) item of the page
(37) item of the page
No yearBuilt Data.
No lotAreaUnit Data.
(38) item of the page
Try to get home details response (1) time.
status code received: 200
No lotAreaUnit Data.
(39) item of the page
No lotAreaUnit Data.
(40) item of the page
No lotAreaUnit Data.
status code received: 200
{'baseUrl': '/easley-sc/2_p/', 'windowTitle': 'Easley Real Estate - Easley SC Homes For Sale | Zillow'}
how many results are we processing? 40
(1) item of the page
(2) item of the page
No lotAreaUnit Data.
(3) item of the page
No yearBuilt Data.
(4) item of the 

In [42]:
#######################################################################################################################

# create a folder called zillow_historical_data_zipcode before run the rest part
# write data out to a csv file
cleaned_scraped_data = remove_duplicates(scraped_data)
write_data_to_csv(cleaned_scraped_data, county_city_zipcode, state, pricetype)

price_history_list, tax_history_list = get_price_tax_history_list(cleaned_scraped_data)
write_historical_data_to_csv(price_history_list, county_city_zipcode, state, 'price', pricetype)
write_historical_data_to_csv(tax_history_list, county_city_zipcode, state, 'tax', pricetype)